In [ ]:
from websockets.sync.client import connect
import json
import pandas as pd

In this example we use the websockets library (https://websockets.readthedocs.io/en/stable/)

This is a super simple way to receive the websocket data, but feel free to use other methods

In [ ]:
data = []

def receive_data():
    max_msg = 100 # Feel free to change this or just let it loop forever. Maybe best to keep it low while testing...
    with connect("ws://192.87.172.71:1338") as websocket:
        count = 0
        while count < max_msg:
            msg = websocket.recv()
            try:
                msg = json.loads(msg)
                #handle_message(msg) # Call function to do the actual data handling
            except json.JSONDecodeError:
                print("Failed to decode json, assuming next packet will be ok...")
                pass
            except Exception as e:
                # Assume something went wrong and stop receiving
                print("Something went horribly wrong!")
                print("Error:", e)
                break
            
            count += 1
            data.append(msg)
        print("Received %d messages!" % count)


# 'Packet' format
The websocket will give you one entry per packet that was received on a receiver.

The packet can have the following fields:

- `address`: The address of the airplane that transmitted this message; you can use the `address` field to match different messages for the same aircraft,
- `altitude`: The altitude (in feet [https://en.wikipedia.org/wiki/Foot_(unit)]) of the airplane,
- `latitude`: The latitude of the airplane,
- `longitude`: The longitude of the airplane,
- `speed`: The speed (in knots [https://en.wikipedia.org/wiki/Knot_(unit)]) of the airplane,
- `heading`: The heading (in degrees) of the airplane,
- `callsign`: The callsign of the airplane
- `timestamp`: The time at which the receiver received the message. Note that this will not be exactly the same on both receivers due to processing delays and the clocks not being exactly in sync!
- `rssi`: The received signal strength indication of the message as determined by the receiver,
- `receiver`: The name of the receiver that received this message (either 'zi-5067' or 'zi-5110')

Note that due to there being two receivers this also means that some packets will be received twice, once on each receiver! You can detect this by noticing that some of the fields will be identical. Only the `timestamp`, `rssi` and `receiver` fields will be different.

There are 4 different 'types' of messages you will need to handle:
- The 'location' message: Has the `altitude`, `latitude` and `longitude` information,
- The 'altitude' message: This only has the `altitude` field,
- The 'speed' message: Has the `speed` and `heading` information,
- The 'callsign' message: Has the `callsign` information.

In [ ]:
def handle_message(msg):
    #print(msg)
    if 'altitude' in msg: # check if this is one of the two types with altitude info
        if msg['altitude'] * 0.3048 >= 10000:
            print("Plane {} is at 10km or higher!".format(msg['address']))

# Creating DataFrame


In [ ]:
df = pd.read_csv('planes_total.csv')  

In [ ]:
# Expand data in loop
count = 0
while count < 150:
    receive_data()
    new_df = pd.DataFrame(data)

    # Drop invalid rows
    new_df = new_df.dropna(subset=['altitude', 'latitude', 'longitude', 'address', 'timestamp'])

    # Merge
    df = pd.concat([df, new_df], ignore_index=True)
    df.drop_duplicates(subset=['address', 'timestamp'], inplace=True)

    # Save updated data
    df.to_csv('planes_total.csv', index=False)

    count += 1

In [ ]:
df_zi_5067 = df[df['receiver'] == 'zi-5067']
df_zi_5110 = df[df['receiver'] == 'zi-5110']

# Calculate the distance between plane and receiver

In [ ]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers

    lat1 = np.radians(lat1) #plane latitude
    lon1 = np.radians(lon1) #plane longitude
    lat2 = np.radians(lat2) #receiver latitude
    lon2 = np.radians(lon2) #receiver longitude

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

#The location of zilverling = (52.239, 6.858)
receiver_lat = 52.239
receiver_lon = 6.858

#Add column 'distance' with a corresponding value
df.loc[:,'distance'] = haversine(df['latitude'], df['longitude'], receiver_lat, receiver_lon)
df_zi_5067.loc[:,'distance'] = haversine(df_zi_5067['latitude'], df_zi_5067['longitude'], receiver_lat, receiver_lon)
df_zi_5110.loc[:,'distance'] = haversine(df_zi_5110['latitude'], df_zi_5110['longitude'], receiver_lat, receiver_lon)

# per-group time-based thinning (for the visibility of plane mark in the map)
- Retain the first recorded row per plane & the next row only if it's at least 0.001 seconds later than the last row we kept for that plane

In [ ]:
df = df.sort_values(by=['address', 'timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'])

first_times = df.groupby('address')['timestamp'].transform('min')

# Function to keep spaced rows per address
def thin_by_time(group, threshold=0.1):
    kept_rows = [group.iloc[0]]  # Always keep the first row
    last_time = group.iloc[0]['timestamp'] #time stamp of the first row

    for i in range(1, len(group)):
        current_row = group.iloc[i]
        time_diff = (current_row['timestamp'] - last_time).total_seconds()
        
        if time_diff >= threshold:
            kept_rows.append(current_row)
            last_time = current_row['timestamp']
    return pd.DataFrame(kept_rows)

# Apply to each address
df_thinned = df.groupby('address', group_keys=False).apply(thin_by_time)

In [ ]:
df_thinned

# Correlation Scatter Plot 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def correlation_check(df, col1, col2):
    corr = df[col1].corr(df[col2])  # Default is Pearson
    
    # Plot
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=df, x=col1, y=col2, hue=col1, palette='RdYlGn', edgecolor = None)
    plt.title(f'Correlation = {corr:.2f}')
    plt.xlabel('{}'.format(col1))
    plt.ylabel('{}'.format(col2))
    plt.grid(True)
    plt.show()

In [ ]:
correlation_check(df, 'distance', 'altitude')

# Save the DataFrames as CSV files

In [ ]:
df.to_csv('planes_total.csv', index=False)
df_zi_5110.to_csv('zi_5110.csv', index=False)
df_zi_5067.to_csv('zi_5067.csv', index=False)

df_dis = df.sort_values(by ="distance", ascending=False)
df_signal = df.sort_values(by="rssi", ascending=False)
df_dis.to_csv('planes_sorted_by_distance.csv') 
df_dis.to_csv('planes_sorted_by_rssi.csv') 

df_zi_5067_dis = df_zi_5067.sort_values(by="distance", ascending=False)
df_zi_5067_signal = df_zi_5067.sort_values(by="rssi", ascending=False)
df_zi_5067_dis.to_csv('zi-5067_sorted_by_distance.csv') 
df_zi_5067_signal.to_csv('zi-5067_sorted_by_rssi.csv') 

df_zi_5110_dis = df_zi_5110.sort_values(by="distance", ascending=False)
df_zi_5110_signal = df_zi_5110.sort_values(by="rssi", ascending=False)
df_zi_5110_dis.to_csv('zi-5110_sorted_by_distance.csv') 
df_zi_5110_signal.to_csv('zi-5110_sorted_by_rssi.csv') 



# Map Visualization 

- marks down the location of each plane
- illustrates the signal strength and altitude by colors

In [ ]:
pip install folium

In [ ]:
map_df = df

In [ ]:
import folium
import folium.plugins as plugins

m = folium.Map(location=[map_df['latitude'].mean(), map_df['longitude'].mean()],tiles = "Stadia.StamenTerrain", zoom_start=8)

# Add airplane markers

def altitude_color(altitude):
    if altitude > 40000:
        return 'white'
    elif altitude > 25000:
        return 'lightblue'
    elif altitude > 10000:
        return 'blue'
    else:
        return 'darkblue'

def signal_color(signal):
    if signal > -60:
        return 'darkgreen'
    elif signal > -80:
        return 'green' 
    elif signal > -100:
        return 'orange' 
    elif signal > -120:
        return 'red'  
    else: 
        return 'darkred'

for _, row in map_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Plane{row['address']},  {row['altitude']} fts",
        icon = plugins.BeautifyIcon(
            icon = 'plane',
            icon_shape = None,
            border_width= 0,
            text_color = altitude_color(row['altitude']),
            background_color = signal_color(row['rssi'])
            )
        ).add_to(m)
    

folium.Marker(
        location=[52.23923592912191, 6.857271194458009],
        popup=f"receiver",
        icon = folium.Icon(color='black', icon_color = 'white', icon ='satellite-dish', prefix = 'fa')
    ).add_to(m)

m.save("airplanes_map.html") 

In [ ]:
m_receiver = folium.Map(location=[map_df['latitude'].mean(), map_df['longitude'].mean()],tiles = "Stadia.StamenTerrain", zoom_start=8)
m_alt = folium.Map(location=[map_df['latitude'].mean(), map_df['longitude'].mean()],tiles = "Stadia.StamenTerrain", zoom_start=8)

for _, row in map_df.iterrows():
    if row['receiver'] == 'zi-5067':
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            popup=f"Plane{row['address']},  {row['altitude']} fts",
            radius=10,
            color="#B379DA", #Purple
            stroke=False,
            fill=True,
            fill_opacity=0.2,
            opacity=0.2
            ).add_to(m_receiver)
    
    else: 
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            popup=f"Plane{row['address']},  {row['altitude']} fts",
            radius=10,
            color="#FAA245", #orange
            stroke=False,
            fill=True,
            fill_opacity=0.2,
            opacity=0.2
            ).add_to(m_receiver)


for _, row in map_df.iterrows():
    if row['altitude'] > 30000:
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            popup=f"Plane{row['address']},  {row['altitude']} fts, {row['distance']}km away",
            radius=12,
            color = '#CC6600',
            weight = 1,
            opacity = 0.5,
            fill=True,
            fill_opacity=0.5,
            fill_color="#ED9121", #carrot orange,
            ).add_to(m_alt)
for _, row in map_df.iterrows():
    if row['altitude'] < 20000:
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            popup=f"Plane{row['address']},  {row['altitude']} fts, {row['distance']}km away",
            radius=12,
            color = '#1569C7',
            opacity = 0.5,
            weight = 1,
            fill =True,
            fill_opacity = 0.5,
            fill_color= '#38ACEC', #Butterfly Blue
            ).add_to(m_alt)
    

folium.Marker(
        location=[52.23923592912191, 6.857271194458009],
        popup=f"receiver",
        icon = folium.Icon(color='black', icon_color = 'white', icon ='satellite-dish', prefix = 'fa')
    ).add_to(m_alt)
folium.Marker(
        location=[52.23923592912191, 6.857271194458009],
        popup=f"receiver",
        icon = folium.Icon(color='black', icon_color = 'white', icon ='satellite-dish', prefix = 'fa')
    ).add_to(m_receiver)


In [ ]:
m_alt

In [ ]:
import folium
import folium.plugins as plugins

m = folium.Map(location=[map_df['latitude'].mean(), map_df['longitude'].mean()],tiles = "Stadia.StamenTerrain", zoom_start=8)
#folium.Map(location=[lat, lon], tiles=tiles, attr=attr, zoom_start=zoom_start)

# Add airplane markers

def altitude_color(altitude):
    if altitude > 40000:
        return 'white'
    elif altitude > 30000:
        return 'lightblue'
    elif altitude > 20000:
        return 'blue'
    else:
        return 'darkblue'

def signal_color(signal):
    if signal > -60:
        return 'darkgreen'
    elif signal > -80:
        return 'green' 
    elif signal > -100:
        return 'orange' 
    elif signal > -120:
        return 'red'  
    else: 
        return 'darkred'

for _, row in map_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Plane{row['address']},  {row['altitude']} fts",
        icon = plugins.BeautifyIcon(
            icon = 'plane',
            icon_shape = None,
            border_width= 0,
            text_color = altitude_color(row['altitude']),
            background_color = signal_color(row['rssi'])
            )
        ).add_to(m)

folium.Marker(
        location=[52.23923592912191, 6.857271194458009],
        popup=f"receiver zi-5067",
        icon = folium.Icon(color='black', icon_color = 'white', icon ='satellite-dish', prefix = 'fa')
    ).add_to(m)
folium.Marker(
        location=[52.23895670441498, 6.85721755027771],
        popup=f"receiver zi-5110",
        icon = folium.Icon(color='black', icon_color = 'white', icon ='satellite-dish', prefix = 'fa')
    ).add_to(m)

m.save("airplanes_map.html")  # or just m in Jupyter Notebook to display

In [ ]:
m

In [ ]:
import folium
from folium import WmsTileLayer
import folium.plugins as plugins
import pandas as pd
from datetime import datetime


data = 'planes_total(in).csv'
df = pd.read_csv(data)

df = df[(df['rssi'] <= -120)|(df['rssi'] >= -60)]


m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()],tiles = "Stadia.StamenTerrain", zoom_start=8)
#folium.Map(location=[lat, lon], tiles=tiles, attr=attr, zoom_start=zoom_start)

wms_url = 'https://service.pdok.nl/rws/ahn/wms/v1_0?'

layer_name = 'dsm_05m'

WmsTileLayer(
    url=wms_url,
    layers=layer_name,
    fmt='image/png',
    transparent=True,
    attr='AHN / PDOK / Rijkswaterstaat',
    overlay=True,
    control=True
).add_to(m)


def altitude_color(altitude):
    if altitude > 40000:
        return 'white'
    elif altitude > 30000:
        return 'lightblue'
    elif altitude > 20000:
        return 'blue'
    else:
        return 'darkblue'

def signal_color(signal):
    if signal > -60:
        return 'darkgreen'
    elif signal > -80:
        return 'green'
    elif signal > -100:
        return 'orange'
    elif signal > -120:
        return 'red'
    else:
        return 'darkred'

for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Plane{row['address']},  {row['altitude']} fts",
        icon = plugins.BeautifyIcon(
            icon = 'plane',
            icon_shape = None,
            border_width= 0,
            text_color = altitude_color(row['altitude']),
            background_color = signal_color(row['rssi'])
            )
        ).add_to(m)

folium.Marker(
        location=[52.23923592912191, 6.857271194458009],
        popup=f"receiver zi-5067",
        icon = folium.Icon(color='black', icon_color = 'white', icon ='satellite-dish', prefix = 'fa')
    ).add_to(m)
folium.Marker(
        location=[52.23895670441498, 6.85721755027771],
        popup=f"receiver zi-5110",
        icon = folium.Icon(color='black', icon_color = 'white', icon ='satellite-dish', prefix = 'fa')
    ).add_to(m)

folium.LayerControl().add_to(m)


m.save("airplanes_map_with_terrain.html")

In [ ]:
m